In [1]:
import numpy as np
import pandas as pd
# import warnings
# %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns
# import glob
# import json
# import tldextract   # Accurately separates a URL's subdomain, domain, and public suffix
# from bs4 import BeautifulSoup
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import roc_auc_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import MultinomialNB
# from xgboost import XGBClassifier
# from sklearn.linear_model import LogisticRegression
# import re
# from sklearn.metrics import roc_curve, roc_auc_score
# from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score,classification_report,f1_score
import os
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk 
# import spacy
# nltk.download('omw-1.4')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# # nltk.download('wordnet')
# from wordcloud import WordCloud, ImageColorGenerator,STOPWORDS
# import gensim
# from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords
# import copy
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords,wordnet
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer,LancasterStemmer
# from sklearn.metrics import f1_score
# from pprint import pprint
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

In [2]:
path = '//gpfs//home//psc21zcu//FakeNews_Detection//Dissertation_Work//input_dataset//fakeddit//all_samples//'
comment_path = '//gpfs//home//psc21zcu//FakeNews_Detection//Dissertation_Work//input_dataset//fakeddit//'

In [3]:
new_comment_path = '//gpfs//home//psc21zcu//FakeNews_Detection//Dissertation_Work//fakeddit_merge//'

In [4]:
# Method to read input data from tsv file into dataframe
def create_dataframe_from_tsv(path, file):
    dataframe = pd.read_csv(os.path.join(path, file), delimiter="\t")
    
    # If comment data loaded, clean column 'Unnamed: 0'
    if "Unnamed: 0" in dataframe.columns:
        dataframe = dataframe.drop(["Unnamed: 0"], 1)
        
    return dataframe

In [5]:
fakeddit_data = pd.read_csv(path+"all_train.tsv", sep='\t',encoding='latin-1')

In [6]:
fakeddit_data.shape

(878218, 20)

In [7]:
fakeddit_data['clean_title'].head(5)

0    my walgreens offbrand mucinex was engraved with the letters mucinex but in a different order
1                                                                                             NaN
2                                                             this concerned sink with a tiny hat
3                                                   hackers leak emails from uae ambassador to us
4                                                                                             NaN
Name: clean_title, dtype: object

In [8]:
for i,col in enumerate(fakeddit_data.columns):
    print(i,'-->',col)

0 --> Unnamed: 0
1 --> Unnamed: 0.1
2 --> Unnamed: 0.1.1
3 --> Unnamed: 0.1.1.1
4 --> author
5 --> clean_title
6 --> created_utc
7 --> domain
8 --> hasImage
9 --> id
10 --> image_url
11 --> linked_submission_id
12 --> num_comments
13 --> score
14 --> subreddit
15 --> title
16 --> upvote_ratio
17 --> 2_way_label
18 --> 3_way_label
19 --> 6_way_label


In [9]:
df_full=fakeddit_data.iloc[:,[9,7,12,13,14,16,5,17]]#['id,domain,num_comments','score',subreddit,'upvote_ratio','clean_title','2_way_label'])
df_full=df_full.iloc[:,:]


In [10]:
df_full.shape

(878218, 8)

In [11]:
df_full.head(1)

,id,domain,num_comments,score,subreddit,upvote_ratio,clean_title,2_way_label
0,awxhir,i.imgur.com,2.0,12,mildlyinteresting,0.84,my walgreens offbrand mucinex was engraved with the letters mucinex but in a different order,1


#### Removing Null values based on Clean Title

In [12]:
df_full.isna().sum()

id                   0
domain          242100
num_comments    242100
score                0
subreddit            0
upvote_ratio    242100
clean_title      75429
2_way_label          0
dtype: int64

In [13]:
df_full = df_full[df_full['clean_title'].notna()]

In [15]:
df_full.shape

(802789, 8)

Here there is around 8 lakh records available in fakeddit all sample data. Each news article contains multiple number of comments which are stored in seperate csv file named "all_comments.csv". This csv file contains 1 crore records. As part of model training , we are planning to consider the news articles those having comments and considering only first 2 comments for each news article with its clean title



### Adding Comment also

In [16]:
# Reading in all comment data
df_comments = create_dataframe_from_tsv(comment_path, "all_comments.tsv")

In [17]:
# Display total number of comment data entries in complete comment tsv-file
print("Comment data contains: " + str(len(df_comments)) + " entries.")

Comment data contains: 10697533 entries.


In [18]:
df_comments.head(1)

,id,author,body,isTopLevel,parent_id,submission_id,ups
0,f4deplg,Haus42,"Scroll, scroll, scroll. Pause. Scroll back up. Check which subreddit this article is in. Nod in approval. Upvote. Continue scrolling.",True,t3_dkdml1,dkdml1,162.0


In [19]:
df_comments.isna().sum()

id                608607
author           1697170
body              609065
isTopLevel        695995
parent_id         696414
submission_id     696414
ups               696414
dtype: int64

In [20]:
df_comments = df_comments[df_comments['body'].notna()]

In [21]:
df_comments.shape

(10088468, 7)

In [22]:
# The associated comments of an original Reddit-Post are linked via
# the (post) id (in respective data set splits) eualling the submission_id in comments data set
df_full.loc[df_full["id"] == df_comments["submission_id"][0]]

,id,domain,num_comments,score,subreddit,upvote_ratio,clean_title,2_way_label
221246,dkdml1,theonion.com,15.0,1491,theonion,0.99,mark zuckerberg announces all of facebooks future decisions will be made by the cube of justice,0


##### Here we combining news article dataframe with the comment dataframe into a single dataframe called merged_comments_df

In [23]:
merged_comments_df = pd.merge(left=df_full[['id','domain','num_comments','subreddit','clean_title','2_way_label']], right=df_comments[['submission_id','body']], how='inner',left_on='id', right_on='submission_id')

In [24]:
merged_comments_df.head(1)

,id,domain,num_comments,subreddit,clean_title,2_way_label,submission_id,body
0,awxhir,i.imgur.com,2.0,mildlyinteresting,my walgreens offbrand mucinex was engraved with the letters mucinex but in a different order,1,awxhir,Does it help with Dyslexia?


In [26]:
merged_comments_df.shape

(7763932, 8)

#### Helper function to combine first 2 comments data for each submission id and clean it and concatenate it with the clean title in merged df

In [25]:
def combine_reviews(df1,filename):
    from tqdm import tqdm
    length = 2048
#     filename='sampledata.tsv'
    for id in tqdm(df1["id"]):
        #print(id)
        # List to append found comments to
        post_comments = []
        # Find corresponding comments via comparison of (post) id to submission_id in comment data
        df_per_post = df1.loc[df1["submission_id"] == str(id)]
        #print(df_per_post.head())
        # If no comments assigned to specific post, assign empty string
        if len(df_per_post) == 0:
            df1.loc[df1.loc[df1["id"] == str(id)].index[0], "comments"] = ""
        else:
            try:
                #Iterate over all found comments --> Handling different indices and iteration counts
                for i in range(df_per_post.index[0], df_per_post.index[0] + len(df_per_post)):
                    # If comment deleted, then skip
                    if df_per_post["body"][i] == "[deleted]":
                        continue
                    # If comment removed, then skip
                    elif df_per_post["body"][i] == "[removed]":
                        continue
                    else:
                        # If true comment found, append to corresponding comment list
                        comment = str(df_per_post["body"][i])
                        post_comments.append(comment)
            # If indices do not match, assign empty string
                # Usually indicated one single comment by moderator regarding
                # general insignificant instruction for subbreddit conduct guidelines
            except KeyError:
                comment = ""
                post_comments.append(comment)
        # If no cases apply, empty string
        if not post_comments:
            post_comments.append("")
        # Concatenate only first 2 found comments
        comments = " ".join(post_comments[:2])
        # If comments exceed given length limit, slice list
        comments = (comments[:length]) if len(comments) > length else comments

        # Implement found comments per post in 17th column of post dataframe
        df1.loc[df1.loc[df1["id"] == str(id)].index[0], "comments"] = comments
    df1 = df1[df1['comments'].notna()]
    df1.drop(['body'], axis=1,inplace=True)
    # Writing df_input split including comments column to tsv file
    print("Writing comment data split to csv file.")
    df1.to_csv(f"{filename}", sep="\t", index=False, header=True) 
    return df1

But the merged_comments_df contains around 77 lakh records with duplicate news ids but with different comments for each news article..


By using the above helper function  **combine_reviews**, we need to create a single dataframe which should contain unique news id column with combined first 2 comments 

Due to huge data size and memory constraint issues, we cannot do this at once.


So I have created 24 different dataframes by splitting  around 3 -4 lakh records for each dataframe and then run the above helper function on each dataframe and filtered  the subset of news ids with valid comments  and saved those in 24 different tsv file formats.


Then combining all these 24 tsv files , created a single dataframe which contains the news articles with valid comments

In [ ]:
# Method to read input data from tsv file into dataframe
def create_dataframe_from_tsv(path, file):
    dataframe = pd.read_csv(os.path.join(path, file), delimiter="\t")
    
    # If comment data loaded, clean column 'Unnamed: 0'
    if "Unnamed: 0" in dataframe.columns:
        dataframe = dataframe.drop(["Unnamed: 0"], 1)
        
    return dataframe

In [27]:
df_1 = pd.read_csv("fakeddit_data_full_comments_1.tsv", sep='\t',encoding='latin-1')
df_2 = pd.read_csv("fakeddit_data_full_comments_2.tsv", sep='\t',encoding='latin-1')
df_3 = pd.read_csv("fakeddit_data_full_comments_3.tsv", sep='\t',encoding='latin-1')
df_4 = pd.read_csv("fakeddit_data_full_comments_4.tsv", sep='\t',encoding='latin-1')
df_5 = pd.read_csv("fakeddit_data_full_comments_5.tsv", sep='\t',encoding='latin-1')
df_6 = pd.read_csv("fakeddit_data_full_comments_6.tsv", sep='\t',encoding='latin-1')
df_7 = pd.read_csv("fakeddit_data_full_comments_7.tsv", sep='\t',encoding='latin-1')
df_8 = pd.read_csv("fakeddit_data_full_comments_8.tsv", sep='\t',encoding='latin-1')
df_9 = pd.read_csv("fakeddit_data_full_comments_9.tsv", sep='\t',encoding='latin-1')
df_10 = pd.read_csv("fakeddit_data_full_comments_10.tsv", sep='\t',encoding='latin-1')
df_11 = pd.read_csv("fakeddit_data_full_comments_11.tsv", sep='\t',encoding='latin-1')
df_12 = pd.read_csv("fakeddit_data_full_comments_12.tsv", sep='\t',encoding='latin-1')
df_13 = pd.read_csv("fakeddit_data_full_comments_13.tsv", sep='\t',encoding='latin-1')
df_14 = pd.read_csv("fakeddit_data_full_comments_14.tsv", sep='\t',encoding='latin-1')
df_15 = pd.read_csv("fakeddit_data_full_comments_15.tsv", sep='\t',encoding='latin-1')
df_16 = pd.read_csv("fakeddit_data_full_comments_16.tsv", sep='\t',encoding='latin-1')
df_17 = pd.read_csv("fakeddit_data_full_comments_17.tsv", sep='\t',encoding='latin-1')
df_18 = pd.read_csv("fakeddit_data_full_comments_18.tsv", sep='\t',encoding='latin-1')
df_19 = pd.read_csv("fakeddit_data_full_comments_19.tsv", sep='\t',encoding='latin-1')
df_20 = pd.read_csv("fakeddit_data_full_comments_20.tsv", sep='\t',encoding='latin-1')
df_21 = pd.read_csv("fakeddit_data_full_comments_21.tsv", sep='\t',encoding='latin-1')
df_22 = pd.read_csv("fakeddit_data_full_comments_22.tsv", sep='\t',encoding='latin-1')
df_23 = pd.read_csv("fakeddit_data_full_comments_23.tsv", sep='\t',encoding='latin-1')
df_24 = pd.read_csv("fakeddit_data_full_comments_24.tsv", sep='\t',encoding='latin-1')

In [28]:
df_list = [df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15,df_16,df_17,df_18,df_19,df_20,df_21,df_22,df_23,df_24]

In [30]:
# Converting dataframe
fakeddit_full_valid_df = pd.concat(df_list).reset_index(drop=True)
total_rows = fakeddit_full_valid_df.shape[0]
print(total_rows)

481781


#### Around 4.8 lakh valid records  are filtered from total rows of 8.8 lakh news articles

In [33]:
##### Saving Combined Dataframe to csv file

fakeddit_full_valid_df.to_csv('//gpfs//home//psc21zcu//FakeNews_Detection//Dissertation_Work//Updated//fakeddit_full_valid_df.csv', index=False)

In [34]:
fakeddit_full_valid_df.head(100)

,id,domain,num_comments,subreddit,clean_title,2_way_label,submission_id,comments
0,awxhir,i.imgur.com,2.0,mildlyinteresting,my walgreens offbrand mucinex was engraved with the letters mucinex but in a different order,1,awxhir,Does it help with Dyslexia? Pretty sure it makes it worse
1,98pbid,i.redd.it,2.0,pareidolia,this concerned sink with a tiny hat,0,98pbid,âDoes this hat make my head look too big?â -too
2,6f2cy5,aljazeera.com,1.0,neutralnews,hackers leak emails from uae ambassador to us,1,6f2cy5,"---- **/r/NeutralNews is a curated space.**\nIn order not to get your comment removed, please familiarize yourself with our [rules on commenting](https://www.reddit.com/r/NeutralNews/wiki/guidelines#wiki_comments_.28good.2C_bad_.26amp.3B_ugly.29) before you participate:\n#Comment Rules\nWe expect the following from all users:\n\n1. Be courteous to other users.\n1. Source your facts.\n1. Put thought into it.\n1. Address the arguments, not the person.\n\n\nIf you see a comment that violates any of these essential rules, click the associated *report* link so mods can attend to it.\nHowever, **please note that the mods will not remove comments or links reported for lack of neutrality**. There is [no neutrality requirement for comments or links](https://www.reddit.com/r/neutralnews/wiki/guidelines#wiki_neutral-ness) in this subreddit â it's only the *space* that's neutral â and a poor source should be countered with evidence from a better one.\n\n*I am a bot, and this action was per..."
3,4xypkv,i.imgur.com,26.0,photoshopbattles,puppy taking in the view,1,4xypkv,"[EVERYTHING THE LIGHT TOUCHES IS OUR KINGDOM](http://i.imgur.com/H1wXuSM.png) Uh guys, you leaving? I thought we were best friends? http://imgur.com/a/chxWw"
4,8gnet9,i.redd.it,2.0,pareidolia,i found a face in my sheet music too,0,8gnet9,That crescendo part at the top also kind of looks like a face. GIcy f secondz we\nTfleswexweggggvggg
5,71lejc,i.imgur.com,1.0,fakealbumcovers,escobar a couple of thoughts,0,71lejc,"Thank you for submitting your post to /r/fakealbumcovers. Please make sure you have read [**the rules in the Wiki**](https://www.reddit.com/r/fakealbumcovers/wiki/rules).\n\nIf you've submitted an album cover or a request, please add a link flair to your post. [**You can learn more about link flairs here.**](https://www.reddit.com/r/fakealbumcovers/wiki/linkflairs)\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/fakealbumcovers) if you have any questions or concerns.*"
6,2vkbtj,independent.ie,7.0,nottheonion,bride and groom exchange vows after fatal shooting at their wedding,1,2vkbtj,"""We planned this weding for more than a year. There's no way we are not doing it..."" Hey OP, did they ever explain what the motives where to why they got shot?"
7,86byl8,timesofisrael.com,577.0,nottheonion,rabbi meat from cloned pig could be kosher for jews to eat with milk,1,86byl8,"Misleading headline: he's saying lab-grown meat is kosher, not meat from cloned pigs. They clarify in the article.\n\n>""In the interview, Cherlow of the Tzohar Rabbinical Organization appears to be talking about meat that is grown artificially in a laboratory from the cells of a pig, rather than meat produced from a live pig whose genetic material comes from a cell from which the pig was cloned.""\n\nSummary of the interpretation: Kosher rules forbid consuming the flesh of a pig, it says nothing about the genetic material. This lab-grown meat is created fully outside of a living animal, and thus is not flesh of a pig, even though it's the same genetic material. By that ""flesh of an animal"" standard, this isn't even meat at all, which is why it's kosher. Technology will eventually allow us to advance so far that the old laws aren't even applicable anymore. \n\nFor example, if you're on a space station very far from a long abandoned Earth, and days and nights are now an artificial con..."
8,bmg5cj,i.redd.it,1.

In [35]:
fakeddit_full_valid_df['2_way_label'].value_counts()

1    332853
0    148928
Name: 2_way_label, dtype: int64